In [1]:
from absbox import API, mkDeal

localAPI = API("http://localhost:8081/", lang="english")


name = "Name"
dates = {
    "cutoff":"2024-08-31","closing":"2024-08-31","firstPay":"2024-09-30",
    "stated": "2027-09-30",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan = ["Mortgage"
            ,{"originBalance": 10925
            ,"originRate": ["fix",0.22]
            ,"originTerm": 11
            ,"freq": "Monthly"
            ,"type": "even"
            ,"originDate": "2024-08-31"}
            ,{"currentBalance": 10925
            ,"currentRate": 0.22
            ,"remainTerm": 11
            ,"status": "current"}]

bullet_loans = ["Loan"
            ,{"originBalance": 5750
            ,"originRate": ["fix",0.22]
            ,"originTerm": 3
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-08-31"}
            ,{"currentBalance": 5750
            ,"currentRate": 0.22
            ,"remainTerm": 3
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan]*100},"pool_bullet": {'assets':[bullet_loans]*10}}

accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal_amortizing": {"balance": 0},
    "collections_principal_bullet": {"balance": 0},
    "revolBuyAcc1": {"balance": 0},
    "revolBuyAcc2": {"balance": 0},
    "dealAcc": {"balance": 0},
}

bonds = {
    "A1": {
        "balance": 920_000.00,
        "rate": 0.13,
        "originBalance": 10_000_000,
        "originRate": 0.13,
        "startDate": "2024-08-31",
        "rateType": {"fix": 0.13, "dayCount": "DC_ACT_365A"},
        "maturityDate": "2027-09-30",
        "bondType": {"Sequential":None},
    },
    "EQ": {
        "balance": 230_000,
        "rate": 0.0,
        "originBalance": 230_000,
        "originRate": 0.0,
        "startDate": "2024-08-31",
        "maturityDate": "2027-09-30",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}

                                

fees = {"utilisation_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A1"),("bondBalance", "A1"),),0.0075],},
                            "feeStart": "2024-12-31"},}
waterfall = {
                    "Accelerated":[
                                ["calcInt", "A1"],
                                ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",475_000)}],
                                ["fundWith", "revolBuyAcc2", "A1",{"formula": ("const",25_000)}],
                                ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",118_750)}],
                                ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",6250)}],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "revolving":[
                                ["calcInt", "A1"],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),)
                                    ,"support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                
                                    ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "amortizing":[
                                ["calcInt", "A1"],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ['transfer','collections_principal_amortizing','dealAcc'],
                                ['transfer','collections_principal_bullet','dealAcc'],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],

                    "cleanUp": [
                            ["sellAsset", ["Current|Defaulted", 1.0, 0], "dealAcc"],
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payPrin", "dealAcc", ["EQ"]],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ],

                    "Defaulted": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ["payPrin", "dealAcc", ["EQ"]],],
                            
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]
defrate3periods = \
   [ ("ratio",("curPoolCollectionStats", None, _*-1, "Defaults"),("curPoolCollectionStats", None, _*-1, "BegBalance"))
   for _ in range(3) ]

trigger = {
            "AfterCollect": {
                "revolving_trigger": {
                    "condition": ["any", [">=", "2026-02-28"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-08-01"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "DefaultRate_trigger": { #Rolling Default rate trigger
            "condition": ["all", 
                          [">=", "2024-12-30"],
                          [("avgRatio",*defrate3periods),">", 0.03167],
            ],
            
            "effects": ("newStatus", "Defaulted"),
            "status": False,
            "curable": False,
        },},
        "BeforeDistribution": {
                "LTV_trigger": {
                    "condition": ["all",
                                  [">", "2024-09-30"],
                                  [
                        (
                            "/",
                            ("bondBalance","A1"),
                            ("sum",
                             ("poolBalance",),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             ),
                        ),
                            ">",
                        0.8,
                    ],],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
            }, 
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Accelerated"),
        }
    
deal = mkDeal(deal_data)
revol_asset1 = amortising_loan
revol_asset2 = bullet_loans

cdr_assump = 0.1

cdr_assump_1 = 0.1

revolvingPool1 = (["constant",revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,None
                                                                   )

                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,None)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

localAPI = API("http://localhost:8081/", lang="english")
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
                              ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, 0, "Defaults"),("curPoolCollectionStats", None, 0, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -1, "Defaults"),("curPoolCollectionStats", None, -1, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -2, "Defaults"),("curPoolCollectionStats", None, -2, "BegBalance")))
                              
                              ),
                              ("call", {"afterDate":"2027-08-01"}),
                    ("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
                                              ]
         ,read=True)



Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.6, server:0.29.8

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.6, server:0.29.8

Warning Message from server:
Bond EQ is not paid off

In [2]:
r['pool']['flow']['pool_amortizing']


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,1092500.00,0.00,0.00,0,0.00,0,0.00,0.220000,None,None,0.00,0,0,0.00,0,0.00
2024-09-30,984620.00,98461.00,19856.00,0,9419.00,0,9419.00,0.220000,None,None,98461.00,0,0,9419.00,0,9419.00
2024-10-31,631005.34,160684.53,30615.04,0,14807.89,0,14807.89,0.220000,None,None,259145.53,0,0,24226.89,0,24226.89
2024-11-30,770987.01,228766.78,41439.79,0,19841.05,0,19841.05,0.220000,None,None,487912.31,0,0,44067.94,0,44067.94
2024-12-31,1786964.81,226747.54,36916.94,0,17937.17,0,17937.17,0.220000,None,None,714659.85,0,0,62005.11,0,62005.11
2025-01-31,1546319.05,224726.73,32468.56,0,15919.03,0,15919.03,0.220000,None,None,939386.58,0,0,77924.14,0,77924.14
2025-02-28,1310847.96,222901.07,28117.45,0,12570.02,0,12570.02,0.220000,None,None,1162287.65,0,0,90494.16,0,90494.16
2025-03-31,1078343.07,220932.54,23818.94,0,11572.35,0,11572.35,0.220000,None,None,1383220.19,0,0,102066.51,0,102066.51
2025-04-30,849948.09,219010.82,19596.34,0,9384.16,0,9384.16,0.220000,None,None,1602231.01,0,0,111450.67,0,111450.67


In [3]:
localAPI = API("http://localhost:8081/", lang="english")
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
                              ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, 0, "Defaults"),("curPoolCollectionStats", None, 0, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -1, "Defaults"),("curPoolCollectionStats", None, -1, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -2, "Defaults"),("curPoolCollectionStats", None, -2, "BegBalance")))
                              
                              ),
                              ("call", {"afterDate":"2027-08-01"}),
                    ("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
                                              ]
         ,read=True)


Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.6, server:0.29.5

Warning Message from server:
Bond EQ is not paid off

In [4]:
r['pool']['flow']['pool_amortizing']


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,1092500.00,0.00,0.00,0,0.00,0,0.00,0.220000,None,None,0.00,0,0,0.00,0,0.00
2024-09-30,984620.00,98461.00,19856.00,0,9419.00,0,9419.00,0.220000,None,None,98461.00,0,0,9419.00,0,9419.00
2024-10-31,1509270.34,160684.53,30615.04,0,14807.89,0,14807.89,0.220000,None,None,259145.53,0,0,24226.89,0,24226.89
2024-11-30,2029398.17,228541.65,41394.39,0,19819.51,0,19819.51,0.220000,None,None,487687.18,0,0,44046.40,0,44046.40
2024-12-31,2543616.65,302390.40,52175.39,0,25175.29,0,25175.29,0.220000,None,None,790077.58,0,0,69221.69,0,69221.69
2025-01-31,3051267.62,382696.36,62954.82,0,30866.30,0,30866.30,0.220000,None,None,1172773.94,0,0,100087.99,0,100087.99
2025-02-28,3553948.60,470291.24,73775.60,0,32976.15,0,32976.15,0.220000,None,None,1643065.18,0,0,133064.14,0,133064.14
2025-03-31,4047622.11,565259.96,84567.26,0,40232.47,0,40232.47,0.220000,None,None,2208325.14,0,0,173296.61,0,173296.61
2025-04-30,4533241.03,668455.49,95361.80,0,45763.51,0,45763.51,0.220000,None,None,2876780.63,0,0,219060.12,0,219060.12


In [5]:
#Experimenting with extra stress feature


cdr_assump = 0.1

cdr_assump_1 = 0.1



defaultFactor = [
 ["2025-09-01", 9],
 ["2025-10-01", 9],
 ["2025-11-01", 9],
 ["2025-12-01", 9],
 ["2026-01-01", 9],
 ["2026-02-01", 9],
 ["2026-03-01", 9],
 ["2026-04-01", 9],
 ["2026-05-01", 9],
 ["2026-06-01", 9],
 ["2026-07-01", 9],
 ["2026-08-01", 9],
 ["2026-09-01", 9],
 ["2026-10-01", 9],
 ["2026-11-01", 9],
 ["2026-12-01", 9],
 ["2027-01-01", 9],
 ["2027-02-01", 9],
 ["2027-03-01", 9],
 ["2027-04-01", 9],
 ["2027-05-01", 9],
 ["2027-06-01", 9],
 ["2027-07-01", 9],
 ["2027-08-01", 9],
 ["2027-09-01", 9]]

defaultFactorEndOfMonth = [
 ["2025-09-30", 9],
 ["2025-10-31", 9],
 ["2025-11-30", 9],
 ["2025-12-31", 9],
 ["2026-01-31", 9],
 ["2026-02-28", 9],
 ["2026-03-31", 9],
 ["2026-04-30", 9],
 ["2026-05-31", 9],
 ["2026-06-30", 9],
 ["2026-07-31", 9],
 ["2026-08-31", 9],
 ["2026-09-30", 9],
 ["2026-10-31", 9],
 ["2026-11-30", 9],
 ["2026-12-31", 9],
 ["2027-01-31", 9],
 ["2027-02-28", 9],
 ["2027-03-31", 9],
 ["2027-04-30", 9],
 ["2027-05-31", 9],
 ["2027-06-30", 9],
 ["2027-07-31", 9],
 ["2027-08-31", 9],
 ["2027-09-30", 9]]


revolvingPool1 = (["constant",revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,defaultFactorEndOfMonth
                                                                   )

                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,defaultFactorEndOfMonth)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,defaultFactorEndOfMonth)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,defaultFactorEndOfMonth)
                   ,None
                   ,None)

localAPI = API("http://localhost:8081/", lang="english")
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
                              ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, 0, "Defaults"),("curPoolCollectionStats", None, 0, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -1, "Defaults"),("curPoolCollectionStats", None, -1, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -2, "Defaults"),("curPoolCollectionStats", None, -2, "BegBalance")))
                              
                              ),
                              ("call", {"afterDate":"2027-08-01"}),
                    ("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
                                              ]
         ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.6, server:0.29.5

Warning Message from server:
Bond EQ is not paid off

In [6]:
r['pool']['flow']['pool_amortizing']
# The default factor isn't applied on the specified dates not sure if that's because I've defined it wrong

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,1092500.00,0.00,0.00,0,0.00,0,0.00,0.220000,None,None,0.00,0,0,0.00,0,0.00
2024-09-30,984620.00,98461.00,19856.00,0,9419.00,0,9419.00,0.220000,None,None,98461.00,0,0,9419.00,0,9419.00
2024-10-31,1509270.34,160684.53,30615.04,0,14807.89,0,14807.89,0.220000,None,None,259145.53,0,0,24226.89,0,24226.89
2024-11-30,2029398.17,228541.65,41394.39,0,19819.51,0,19819.51,0.220000,None,None,487687.18,0,0,44046.40,0,44046.40
2024-12-31,2543616.65,302390.40,52175.39,0,25175.29,0,25175.29,0.220000,None,None,790077.58,0,0,69221.69,0,69221.69
2025-01-31,3051267.62,382696.36,62954.82,0,30866.30,0,30866.30,0.220000,None,None,1172773.94,0,0,100087.99,0,100087.99
2025-02-28,3553948.60,470291.24,73775.60,0,32976.15,0,32976.15,0.220000,None,None,1643065.18,0,0,133064.14,0,133064.14
2025-03-31,4047622.11,565259.96,84567.26,0,40232.47,0,40232.47,0.220000,None,None,2208325.14,0,0,173296.61,0,173296.61
2025-04-30,4533241.03,668455.49,95361.80,0,45763.51,0,45763.51,0.220000,None,None,2876780.63,0,0,219060.12,0,219060.12
